In [ ]:
## Imports and setting up GDAL environment variables
import os, sys
import requests as rq
import json

import logging

import geopandas as gpd

import numpy as np

from PIL import Image

import shapely
import shapely.wkt
from shapely.ops import transform as shapely_transform
from shapely.geometry import shape, MultiPolygon

import pyproj
from pyproj import Proj

import rasterio as rio
from rasterio.mask import mask

import folium
from folium import plugins

In [ ]:
# setting env variables
os.environ['GDAL_HTTP_COOKIEFILE'] = '~/cookies.txt'
os.environ['GDAL_HTTP_COOKIEJAR'] = '~/cookies.txt'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'YES'
os.environ['CPL_VSIL_CURL_ALLOWED_EXTENSIONS'] ='TIF'
np.seterr(divide='ignore', invalid='ignore')

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
# functions

def add_geojson_to_map(geojson, map):
    folium.GeoJson(geojson['geometry'],
        name = geojson['properties']['id'],
        zoom_on_click=True,
        style_function=styles).add_to(map)

def add_array_to_map(array,bounds,name):
    folium.raster_layers.ImageOverlay(
        image=array,
        name=name,
        opacity=1,
        bounds= boundary
    ).add_to(m)

def get_wgs_coords(lat,lon,crs):

    inProj = Proj(crs)
    outProj = Proj('epsg:4326')
    x1,y1 = lon,lat
    x2,y2 = transform(inProj,outProj,x1,y1)
    return x2,y2

In [ ]:
# keeping a dictionary of custom base maps for folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}


styles =lambda feature: {
        "fillColor": "blue",
        "color": "white",
        "weight": 2,
        "dashArray": "5, 5"}

In [ ]:
# Reading given KML files, and appending them to one dataframe

k15_df = gpd.read_file('k15.kml', driver='KML')
k26_df = gpd.read_file('k26.kml', driver='KML')

kml_df = k15_df.append(k26_df)

kml_df

In [ ]:
# getting bounding coords of both farms

outer_bounds = kml_df.total_bounds.tolist()
outer_bounds

## using STAC endpoint of Sentinel 2 to get requried images

In [ ]:
from pystac_client import Client

catalog = Client.open("https://earth-search.aws.element84.com/v0")


mysearch = catalog.search(
    collections=['sentinel-s2-l2a-cogs'], 
    bbox= outer_bounds, 
    datetime="2019-09-01/2020-03-30")
print(f"{mysearch.matched()} items found")

In [ ]:
import stackstac

In [ ]:
## using stackstac to create a Dask Xarray from STAC metadata

%time
stack = stackstac.stack(mysearch.items_as_collection(),bounds_latlon=outer_bounds)

In [ ]:
stack

In [ ]:
#filtering thru the dataset for required bands and cloud cover, and making required Indices

lowcloud = stack[stack["eo:cloud_cover"] < 20]

nir, red, swir = lowcloud.sel(band="B08"), lowcloud.sel(band="B04"), lowcloud.sel(band="B11")

ndvi = (nir - red) / (nir + red)
ndmi = (nir - swir) / (nir + swir)

## Using Dask for computation, makes large area/long time series calculations faster

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(memory_limit='1GB')
client = Client(cluster)

In [ ]:
client

In [ ]:
monthly_ndvi = ndvi.resample(time="M").mean(dim="time")

In [ ]:
monthly_ndmi = ndmi.resample(time="M").mean(dim="time")

In [ ]:
m_ndmi = monthly_ndmi.compute()

In [ ]:
m_ndmi.shape

In [ ]:
m_ndvi = monthly_ndvi.compute()

In [ ]:
m_ndvi.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(m_ndvi[0])

### adding metadata to Xarray to write better geotiffs

In [ ]:
import rioxarray

In [ ]:
m_ndmi.attrs['nodata'] = np.nan
m_ndvi.attrs['nodata'] = np.nan

m_ndmi.attrs['crs'] = m_ndmi.rio.crs
m_ndvi.attrs['crs'] = m_ndvi.rio.crs

In [ ]:
m_ndvi.rio.to_raster('monthly_ndvi.tif')

In [ ]:
m_ndmi.rio.to_raster('monthly_ndmi.tif')

## Prepping for PNG creation

In [ ]:
## reseting index to get correct index numbering

kmldf = kml_df.reset_index()
del kmldf['index']

In [ ]:
# converting KML WGS84 coords to UTM zone of image

kmldf_utm = kmldf.to_crs("EPSG:"+str(m_ndvi.epsg.values.tolist()))


#simplifying the output to make clipping neater

kml_sim = kmldf_utm.simplify(tolerance=0.5,preserve_topology=False)


# converting to geojson for use in rasterio

geoj = json.loads(kml_sim.geometry.to_json())

In [ ]:
ndvi_ds = rio.open('monthly_ndvi.tif')
ndmi_ds = rio.open('monthly_ndmi.tif')

In [ ]:
month_lookup = {'0':'2019_10','1':'2019_11','2':'2019_12','3':'2020_01','4':'2020_02','5':'2020_03'}
farm_loookup = {'0':'farm_15','1':'farm_26'}

In [ ]:
if not os.path.exists('./farm_15/'):
    os.makedirs('./farm_15/')

if not os.path.exists('./farm_26/'):
    os.makedirs('./farm_26/')

In [ ]:
kwargs = ndvi_ds.profile
kwargs['count'] = 1
kwargs['nodata'] = 0
kwargs['driver'] = 'PNG'
kwargs['dtype'] = 'uint8'
del kwargs['crs']
del kwargs['transform']
del kwargs['tiled']
del kwargs['interleave']

In [ ]:
kwargs

## Creating Colored PNGs for output for both Vegetation and Water stress

### The calculation for Vegetation stress is done using the median NDVI value of a month, and highlighting all pixels that lower than "median - 10% median"

### The calculation for Water stress is done using the median NDMI value of a month, and highlighting all pixels that lower than "median - 60% median"

In [ ]:
for x in range(len(geoj['features'])):
    ndvi_clip, _ = mask(ndvi_ds,[geoj['features'][x]['geometry']],crop=True)
    ndmi_clip, _ = mask(ndmi_ds,[geoj['features'][x]['geometry']],crop=True)
    
    for i in range(ndvi_clip.shape[0]):
        
        ndvi_stress = np.where(ndvi_clip[i] < np.nanmedian(ndvi_clip[i])-(0.1*np.nanmedian(ndvi_clip[i])),1,0)
        
        with rio.open(f'./{farm_loookup[str(x)]}/ndvi_stress_{month_lookup[str(i)]}.png', 'w', **kwargs) as d:
            d.write(ndvi_stress,1)
        
        os.system(f"gdaldem color-relief -alpha './{farm_loookup[str(x)]}/ndvi_stress_{month_lookup[str(i)]}.png' ./color_ndvi.txt './{farm_loookup[str(x)]}/ndvi_stress_{month_lookup[str(i)]}_color.png'")

        if np.nanmedian(ndmi_clip[i]) > 0:
            ndmi_stress = np.where(ndmi_clip[i] < np.nanmedian(ndmi_clip[i])-(0.6*np.nanmedian(ndmi_clip[i])),1,0)
        else:
            ndmi_stress = np.where(ndmi_clip[i] < np.nanmedian(ndmi_clip[i])+(0.6*np.nanmedian(ndmi_clip[i])),1,0)
        
        with rio.open(f'./{farm_loookup[str(x)]}/ndmi_stress_{month_lookup[str(i)]}.png', 'w', **kwargs) as d:
            d.write(ndmi_stress,1)
        
        os.system(f"gdaldem color-relief -alpha './{farm_loookup[str(x)]}/ndmi_stress_{month_lookup[str(i)]}.png' ./color_ndmi.txt './{farm_loookup[str(x)]}/ndmi_stress_{month_lookup[str(i)]}_color.png'")

## Finally, visualizing Farm monthly health

In [ ]:
import glob

m = folium.Map(location=[kmldf.centroid.y[0],kmldf.centroid.x[0]], tiles='Stamen Terrain', zoom_start=15)

for x in range(len(geoj['features'])):
    
    bottom, left, top, right = kmldf.bounds.minx[x], kmldf.bounds.miny[x], kmldf.bounds.maxx[x], kmldf.bounds.maxy[x]
    
    boundary = [[left, bottom],[right, top]]
    
    pngs = sorted(glob.glob(f'./{farm_loookup[str(x)]}/*color.png'))
    
    for png in pngs:
        
        png_array = np.array(Image.open(png))

        add_array_to_map(png_array, boundary, f"{farm_loookup[str(x)]}_{png.split('/')[2].split('.')[0]}")

basemaps['Google Maps'].add_to(m)
basemaps['Google Satellite Hybrid'].add_to(m)

m.add_child(folium.LayerControl())

m